# 关于层和块

## 自定义块

在下面的代码片段中，我们从零开始编写一个块。它包含一个多层感知机，其具有256个隐藏单元的隐藏层和一
个10维输出层。注意，下面的MLP类继承了表示块的类。我们的实现只需要提供我们自己的构造函数（Python中
的__init__函数）和前向传播函数。

In [2]:
import torch
from torch import nn
from torch.nn import functional as F

# 定义一个多层感知机（MLP）类，继承自 nn.Module
class MLP(nn.Module):
    # 初始化方法，用于定义模型的层
    def __init__(self):
        # 调用父类 nn.Module 的初始化方法
        super().__init__()
        
        # 定义隐藏层：输入特征数为 20，输出特征数为 256
        self.hidden = nn.Linear(20, 256)
        
        # 定义输出层：输入特征数为 256，输出特征数为 10
        self.out = nn.Linear(256, 10)

    # 定义前向传播方法，描述如何根据输入计算输出
    def forward(self, X):
        # 将输入 X 传递给隐藏层，并应用 ReLU 激活函数
        hidden_output = F.relu(self.hidden(X))
        
        # 将隐藏层的输出传递给输出层，得到最终的输出
        output = self.out(hidden_output)
        
        # 返回输出结果
        return output

接下来进行一个实例化

In [3]:
net = MLP()
X = torch.rand(2, 20)
net(X)

tensor([[ 0.0196, -0.1717, -0.0400, -0.0408, -0.1197,  0.1120, -0.0521, -0.1820,
          0.0439,  0.0218],
        [ 0.0612, -0.1745, -0.0023, -0.1297, -0.1790,  0.0759, -0.2485, -0.2383,
          0.1429,  0.0845]], grad_fn=<AddmmBackward0>)

## 顺序块

主要是Sequential类的工作模式，尝试构建的简化的MySequential，我们只需要定义两个关键函数：
1. 一种将块逐个追加到列表中的函数；
2. 一种前向传播函数，用于将输入按追加块的顺序传递给块组成的“链条”。

以下是相关代码

In [4]:
# 自定义顺序容器类 MySequential
class MySequential(nn.Module):
    def __init__(self, *args):
        super().__init__()
        # 遍历传入的模块，并将它们添加到 _modules 中
        for idx, module in enumerate(args):
            # _modules 是 nn.Module 的成员变量，类型为 OrderedDict
            self._modules[str(idx)] = module

    def forward(self, X):
        # 按顺序遍历 _modules 中的模块，并依次调用
        for block in self._modules.values():
            X = block(X)
        return X

实现实例并进行测试

In [5]:
net = MySequential(nn.Linear(20, 256), nn.ReLU(), nn.Linear(256, 10))
net(X)

tensor([[ 0.0284, -0.0355,  0.1432,  0.3371, -0.0323,  0.0877, -0.0484,  0.0901,
         -0.1203, -0.1952],
        [ 0.0090,  0.0335,  0.1910,  0.4235, -0.0527,  0.0009, -0.0274,  0.1611,
         -0.1620, -0.1964]], grad_fn=<AddmmBackward0>)

## 在前向传播函数中执行代码

`Sequential` 类极大地简化了模型的构建过程，允许我们通过组合现有模块来创建新的架构，而无需手动定义新的类。然而，并非所有架构都遵循简单的顺序结构。当需要更高的灵活性时，我们必须自定义模型块。例如，我们可能希望在前向传播过程中引入 Python 的控制流（如条件判断或循环），或者执行一些自定义的数学运算，而不仅仅是依赖预定义的神经网络层。

到目前为止，我们讨论的网络中的所有操作都作用于网络的激活值或可训练参数。然而，在某些情况下，我们可能需要引入一些既不是前一层输出也不是可训练参数的项，这些项被称为**常数参数**（constant parameter）。例如，假设我们需要实现一个计算函数 $ f(x, w) = c \cdot w^\top x $ 的层，其中 $ x $ 是输入，$ w $ 是可训练参数，而 $ c $ 是一个在优化过程中不会更新的固定常量。为了实现这样的功能，我们可以定义一个自定义类 `FixedHiddenMLP`。

In [8]:
class FixedHiddenMLP(nn.Module):
    def __init__(self):
        super().__init__()
        # 不计算梯度的随机权重参数。因此其在训练期间保持不变
        self.rand_weight = torch.rand((20, 20), requires_grad=False)
        self.linear = nn.Linear(20, 20)
        
    def forward(self, X):
        X = self.linear(X)
        # 使用创建的常量参数以及relu和mm函数
        X = F.relu(torch.mm(X, self.rand_weight) + 1)
        # 复用全连接层。这相当于两个全连接层共享参数
        X = self.linear(X)
        # 控制流
        while X.abs().sum() > 1:
            X /= 2
        return X.sum()

在这个FixedHiddenMLP模型中，我们实现了一个隐藏层，其权重（self.rand_weight）在实例化时被随机初
始化，之后为常量。这个权重不是一个模型参数，因此它永远不会被反向传播更新。然后，神经网络将这个
固定层的输出通过一个全连接层。

在返回输出之前，模型做了一些不寻常的事情：它运行了一个while循环，在L1范数大于1的条件下，将
输出向量除以2，直到它满足条件为止。

In [9]:
net = FixedHiddenMLP()
net(X)

tensor(0.0399, grad_fn=<SumBackward0>)

我们可以混合搭配各种组合块的方法。在下面的例子中，我们以一些想到的方法嵌套块。

In [15]:
class NestMLP(nn.Module):
    def __init__(self):
        super().__init__()
        self.net = nn.Sequential(nn.Linear(20, 64), nn.ReLU(),
                                 nn.Linear(64, 32), nn.ReLU())
        self.linear = nn.Linear(32, 16)
        
    def forward(self, X):
        return self.linear(self.net(X))
        
chimera = nn.Sequential(NestMLP(), nn.Linear(16, 20), FixedHiddenMLP())
chimera(X)

tensor(0.1549, grad_fn=<SumBackward0>)

# 关于参数管理

In [16]:
import torch
from torch import nn
net = nn.Sequential(nn.Linear(4, 8), nn.ReLU(), nn.Linear(8, 1))
X = torch.rand(size=(2, 4))
net(X)

tensor([[0.1701],
        [0.0988]], grad_fn=<AddmmBackward0>)

## 参数访问
我们从已有模型中访问参数。当通过Sequential类定义模型时，我们可以通过索引来访问模型的任意层。这
就像模型是一个列表一样，每层的参数都在其属性中。如下所示，我们可以检查第二个全连接层的参数。

In [17]:
print(net[2].state_dict())

OrderedDict([('weight', tensor([[ 0.1586,  0.1788, -0.3422, -0.2022, -0.0853,  0.0258, -0.1181,  0.2561]])), ('bias', tensor([0.1633]))])


### 目标参数

In [18]:
print(type(net[2].bias))
print(net[2].bias)
print(net[2].bias.data)

<class 'torch.nn.parameter.Parameter'>
Parameter containing:
tensor([0.1633], requires_grad=True)
tensor([0.1633])


In [19]:
net[2].weight.grad == None

True

### 一次性访问所有参数

当我们需要对所有参数执行操作时，逐个访问它们可能会很麻烦。当我们处理更复杂的块（例如，嵌套块）
时，情况可能会变得特别复杂，因为我们需要递归整个树来提取每个子块的参数。下面，我们将通过演示来
比较访问第一个全连接层的参数和访问所有层。

In [20]:
print(*[(name, param.shape) for name, param in net[0].named_parameters()])
print(*[(name, param.shape) for name, param in net.named_parameters()])

('weight', torch.Size([8, 4])) ('bias', torch.Size([8]))
('0.weight', torch.Size([8, 4])) ('0.bias', torch.Size([8])) ('2.weight', torch.Size([1, 8])) ('2.bias', torch.Size([1]))


这为我们提供了另一种访问网络参数的方式

In [21]:
net.state_dict()['2.bias'].data

tensor([0.1633])

### 从嵌套块收集参数

In [22]:
def block1():
    return nn.Sequential(nn.Linear(4, 8), nn.ReLU(),
                         nn.Linear(8, 4), nn.ReLU())

def block2():
    net = nn.Sequential()
    for i in range(4):
        # 在这里嵌套
        net.add_module(f'block {i}', block1())
    return net

rgnet = nn.Sequential(block2(), nn.Linear(4, 1))
rgnet(X)

tensor([[-0.2224],
        [-0.2224]], grad_fn=<AddmmBackward0>)

设计了网络后，我们看看它是如何工作的。

In [23]:
print(rgnet)

Sequential(
  (0): Sequential(
    (block 0): Sequential(
      (0): Linear(in_features=4, out_features=8, bias=True)
      (1): ReLU()
      (2): Linear(in_features=8, out_features=4, bias=True)
      (3): ReLU()
    )
    (block 1): Sequential(
      (0): Linear(in_features=4, out_features=8, bias=True)
      (1): ReLU()
      (2): Linear(in_features=8, out_features=4, bias=True)
      (3): ReLU()
    )
    (block 2): Sequential(
      (0): Linear(in_features=4, out_features=8, bias=True)
      (1): ReLU()
      (2): Linear(in_features=8, out_features=4, bias=True)
      (3): ReLU()
    )
    (block 3): Sequential(
      (0): Linear(in_features=4, out_features=8, bias=True)
      (1): ReLU()
      (2): Linear(in_features=8, out_features=4, bias=True)
      (3): ReLU()
    )
  )
  (1): Linear(in_features=4, out_features=1, bias=True)
)


因为层是分层嵌套的，所以我们也可以像通过嵌套列表索引一样访问它们。下面，我们访问第一个主要的块
中、第二个子块的第一层的偏置项。

In [24]:
rgnet[0][1][0].bias.data

tensor([-0.1353,  0.2643, -0.0047, -0.3632,  0.3044,  0.0696,  0.4317,  0.1364])

## 参数初始化

深度学习框架提供默认随机初始化，也允许我们创建自定义初始化方法，满足我们通过其他规则实现初始化权重。

默认情况下，PyTorch会根据一个范围均匀地初始化权重和偏置矩阵，这个范围是根据输入和输出维度计算出的。PyTorch的nn.init模块提供了多种预置初始化方法。

### 内置初始化

In [25]:
# 定义初始化函数
def init_normal(m):
    """
    初始化神经网络中的线性层（nn.Linear）的权重和偏置。
    权重使用正态分布初始化，偏置使用零初始化。
    """
    if type(m) == nn.Linear:  # 检查模块是否为线性层
        nn.init.normal_(m.weight, mean=0, std=0.01)  # 初始化权重：均值为 0，标准差为 0.01
        nn.init.zeros_(m.bias)  # 初始化偏置：值为 0
        
net.apply(init_normal)
net[0].weight.data[0], net[0].bias.data[0]

(tensor([-0.0078,  0.0109,  0.0045,  0.0040]), tensor(0.))

我们还可以将所有参数初始化为给定的常数，比如初始化为1。

def init_constant(m):
    if type(m) == nn.Linear:
        nn.init.constant_(m.weight, 1)
        nn.init.zeros_(m.bias)


net.apply(init_constant)
net[0].weight.data[0], net[0].bias.data[0]

我们还可以对某些块应用不同的初始化方法。例如，下面我们使用Xavier初始化方法初始化第一个神经网络
层，然后将第三个神经网络层初始化为常量值42。

In [28]:
def init_xavier(m):
    if type(m) == nn.Linear:
        nn.init.xavier_uniform_(m.weight)
        
def init_42(m):
    if type(m) == nn.Linear:
        nn.init.constant_(m.weight, 42)
        
net[0].apply(init_xavier)
net[2].apply(init_42)

print(net[0].weight.data[0])
print(net[2].weight.data)

tensor([-0.6525, -0.0530, -0.5031,  0.4587])
tensor([[42., 42., 42., 42., 42., 42., 42., 42.]])


### 自定义的初始化

In [34]:
def my_init(m):
    if type(m) == nn.Linear:
        print("Init", *[(name, param.shape)
                        for name, param in m.named_parameters()][0])
        nn.init.uniform_(m.weight, -10, 10)
        m.weight.data *= m.weight.data.abs() >= 5

net.apply(my_init)
net[0].weight[:2]

Init weight torch.Size([8, 4])
Init weight torch.Size([1, 8])


tensor([[ 0.0000,  9.3591,  6.9189, -0.0000],
        [-0.0000, -7.0772,  5.8547,  0.0000]], grad_fn=<SliceBackward0>)

In [35]:
net[0].weight.data[:] += 1
net[0].weight.data[0, 0] = 42
net[0].weight.data[0]

tensor([42.0000, 10.3591,  7.9189,  1.0000])

## 参数绑定

有时我们希望在多个层间共享参数：我们可以定义一个稠密层，然后使用它的参数来设置另一个层的参数。

In [36]:
# 我们需要给共享层一个名称，以便可以引用它的参数
shared = nn.Linear(8, 8)
net = nn.Sequential(nn.Linear(4, 8), nn.ReLU(),
                    shared, nn.ReLU(),
                    shared, nn.ReLU(),
                    nn.Linear(8, 1))

net(X)
# 检查参数是否相同
print(net[2].weight.data[0] == net[4].weight.data[0])
net[2].weight.data[0, 0] = 100
# 确保它们实际上是同一个对象，而不只是有相同的值
print(net[2].weight.data[0] == net[4].weight.data[0])

tensor([True, True, True, True, True, True, True, True])
tensor([True, True, True, True, True, True, True, True])


# 延后初始化



## 1. **核心思想**
- **延迟初始化**：在模型定义时，不立即初始化参数（如权重和偏置），而是记录参数的结构。
- **动态推断**：在第一次前向传播时，根据输入数据的形状（如批量大小和特征维度）推断参数的实际形状并初始化。

---

## 2. **数学表示**
对于一个线性层 $y = Wx + b$：
- 输入 $x$ 的形状为 $(batch\_size, in\_features)$。
- 权重 $W$ 的形状为 $(out\_features, in\_features)$。
- 偏置 $b$ 的形状为 $(out\_features,)$。

在延后初始化中：
- 模型定义时，$W$ 和 $b$ 的形状未知。
- 第一次前向传播时，根据输入 $x$ 的形状动态推断 $W$ 和 $b$ 的形状并初始化。

---

## 3. **优点**
1. **灵活性**：支持动态输入形状，适用于不同任务和数据集。
2. **节省内存**：在模型定义时不立即分配参数的内存，减少内存占用。
3. **简化代码**：无需手动指定输入形状，代码更简洁。

---

## 4. **缺点**
1. **调试困难**：参数初始化被延迟，调试时难以确定参数的实际形状。
2. **性能开销**：第一次前向传播时需要动态推断参数形状并初始化，可能增加计算开销。

---

## 5. **适用场景**
- 输入数据的形状在模型定义时未知（如动态批量大小或特征维度）。
- 需要灵活处理不同输入形状的任务。
｜

# 关于自定义层

深度学习成功背后的一个因素是神经网络的灵活性：我们可以用创造性的方式组合不同的层，从而设计出适
用于各种任务的架构。例如，研究人员发明了专门用于处理图像、文本、序列数据和执行动态规划的层。有时
我们会遇到或要自己发明一个现在在深度学习框架中还不存在的层。在这些情况下，必须构建自定义层。本
节将展示如何构建自定义层。

## 不带参数的层

下面的CenteredLayer类要从其输入中减去均值。要构建它，我们只需继承基础层类并实现前向传播功能。

In [37]:
import torch
import torch.nn.functional as F
from torch import nn

class CenteredLayer(nn.Module):
    def __init__(self):
        super().__init__()
        
    def forward(self, X):
        return X - X.mean()

们向该层提供一些数据，验证它是否能按预期工作。

In [38]:
layer = CenteredLayer()
layer(torch.FloatTensor([1, 2, 3, 4, 5]))

tensor([-2., -1.,  0.,  1.,  2.])

现在可以将层作为组件合并到更复杂的模型中。

In [39]:
net = nn.Sequential(nn.Linear(8, 128), CenteredLayer())

作为额外的健全性检查，我们可以在向该网络发送随机数据后，检查均值是否为0。由于我们处理的是浮点
数，因为存储精度的原因，我们仍然可能会看到一个非常小的非零数。

In [40]:
Y = net(torch.rand(4, 8))
Y.mean()

tensor(-1.8626e-09, grad_fn=<MeanBackward0>)

## 带参数的层

们继续定义具有参数的层，这些参数可以通过训练进行调整。我们可以使用内置函数来创建参数，这些函数提供一些基本的管理功能。比如管理访问、初始化、共享、保存和加载模型参数。这样做的好处之一是：我们不需要为每个自定义层编写自定义的序列化程序。

现在，让我们实现自定义版本的全连接层。回想一下，该层需要两个参数，一个用于表示权重，另一个用于表示偏置项。在此实现中，我们使用修正线性单元作为激活函数。该层需要输入参数：in_units和units，分别表示输入数和输出数。

In [41]:
import torch
import torch.nn.functional as F
from torch import nn

class MyLinear(nn.Module):
    def __init__(self, in_units, units):
        """
        初始化函数，定义层的参数。
        
        参数:
        - in_units: 输入特征的维度（输入神经元数量）。
        - units: 输出特征的维度（输出神经元数量）。
        """
        super().__init__()  # 调用父类 nn.Module 的初始化方法
        
        # 定义权重参数，形状为 (in_units, units)，并用正态分布随机初始化
        self.weight = nn.Parameter(torch.randn(in_units, units))
        
        # 定义偏置参数，形状为 (units,)，并用正态分布随机初始化
        self.bias = nn.Parameter(torch.randn(units,))
    
    def forward(self, X):
        """
        前向传播函数，定义输入数据如何通过该层进行计算。
        
        参数:
        - X: 输入数据，形状为 (batch_size, in_units)。
        
        返回:
        - 输出数据，形状为 (batch_size, units)。
        """
        # 计算线性变换：X * weight + bias
        linear = torch.matmul(X, self.weight) + self.bias
        
        # 对线性变换的结果应用 ReLU 激活函数
        return F.relu(linear)

接下来，我们实例化MyLinear类并访问其模型参数

In [43]:
linear = MyLinear(5, 3)
linear.weight

Parameter containing:
tensor([[-0.4568,  0.1213,  0.1145],
        [ 0.4236,  0.3502,  0.1224],
        [-0.5629,  0.0027, -1.1536],
        [ 0.9880, -0.1309, -0.3921],
        [-0.1762, -0.3304,  0.3029]], requires_grad=True)

我们可以使用自定义层直接执行前向传播计算。

In [44]:
linear(torch.rand(2, 5))

tensor([[0.0845, 0.0263, 0.0000],
        [0.0000, 0.0000, 0.0000]], grad_fn=<ReluBackward0>)

我们还可以使用自定义层构建模型，就像使用内置的全连接层一样使用自定义层。

In [45]:
net = nn.Sequential(MyLinear(64, 8), MyLinear(8, 1))
net(torch.rand(2, 64))

tensor([[0.8360],
        [0.0000]], grad_fn=<ReluBackward0>)

# 关于读写文件

## 加载和保存张量
对于单个张量，我们可以直接调用load和save函数分别读写它们。这两个函数都要求我们提供一个名称，save要
求将要保存的变量作为输入。

In [52]:
import torch
from torch import nn
from torch.nn import functional as F
x = torch.arange(4)
torch.save(x, 'x-file')

此时我们可以将数据回读

In [53]:
x2 = torch.load('x-file')
x2

/var/folders/82/c4qjdkd94t990kmb846tzgc40000gn/T/ipykernel_73705/1444019752.py:1: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  x2 = torch.load('x-file')


tensor([0, 1, 2, 3])

我们可以存储一个张量列表，然后把它们读回内存。

In [54]:
y = torch.zeros(4)
torch.save([x, y],'x-files')
x2, y2 = torch.load('x-files')
(x2, y2)

/var/folders/82/c4qjdkd94t990kmb846tzgc40000gn/T/ipykernel_73705/2924237495.py:3: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  x2, y2 = torch.load('x-files')


(tensor([0, 1, 2, 3]), tensor([0., 0., 0., 0.]))

我们也可以写入或读取从字符串映射到张量的字典。当我们要读取或写入模型中的所有权重时，这很方便。

In [55]:
mydict = {'x': x, 'y': y}
torch.save(mydict, 'mydict')
mydict2 = torch.load('mydict')
mydict2

/var/folders/82/c4qjdkd94t990kmb846tzgc40000gn/T/ipykernel_73705/2269971588.py:3: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  mydict2 = torch.load('mydict')


{'x': tensor([0, 1, 2, 3]), 'y': tensor([0., 0., 0., 0.])}

## 加载和保存模型参数

保存单个权重向量（或其他张量）确实有用，但是如果我们想保存整个模型，并在以后加载它们，单独保存
每个向量则会变得很麻烦。毕竟，我们可能有数百个参数散布在各处。因此，深度学习框架提供了内置函数
来保存和加载整个网络。需要注意的一个重要细节是，这将保存模型的参数而不是保存整个模型。例如，如
果我们有一个3层多层感知机，我们需要单独指定架构。因为模型本身可以包含任意代码，所以模型本身难以
序列化。因此，为了恢复模型，我们需要用代码生成架构，然后从磁盘加载参数。让我们从熟悉的多层感知
机开始尝试一下。

In [56]:
class MLP(nn.Module):
    def __init__(self):
        super().__init__()
        self.hidden = nn.Linear(20, 256)
        self.output = nn.Linear(256, 10)
    def forward(self, x):
        return self.output(F.relu(self.hidden(x)))

net = MLP()
X = torch.randn(size=(2, 20))
Y = net(X)

接下来将模型的参数存储在一个叫做“mlp.params”的文件中。

In [57]:
torch.save(net.state_dict(), 'mlp.params')

为了恢复模型，我们实例化了原始多层感知机模型的一个备份。这里我们不需要随机初始化模型参数，而是
直接读取文件中存储的参数。

In [58]:
clone = MLP()
clone.load_state_dict(torch.load('mlp.params'))
clone.eval()

/var/folders/82/c4qjdkd94t990kmb846tzgc40000gn/T/ipykernel_73705/2014828273.py:2: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  clone.load_state_dict(torch.load('mlp.params'

MLP(
  (hidden): Linear(in_features=20, out_features=256, bias=True)
  (output): Linear(in_features=256, out_features=10, bias=True)
)

由于两个实例具有相同的模型参数，在输入相同的X时，两个实例的计算结果应该相同。让我们来验证一下。

In [59]:
Y_clone = clone(X)
Y_clone == Y

tensor([[True, True, True, True, True, True, True, True, True, True],
        [True, True, True, True, True, True, True, True, True, True]])